In [1]:
print("The program started") #to notify that the program started running
import pandas as pd
import pandapower as pp
import numpy as np
import pandapower.networks as pn

net = pn.GBreducednetwork() #importing the database as the network for pandapower

results=pd.DataFrame() #create a new dataframe called "results" for selecting the critical lines
results["n_overload"]="" #add a new column in the "results" for the number of overloaded lines
results["excess_overload"]="" #add a new column in the "results" for the total overload excess
results["overload_per_cable"]="" 

net.load['p_mw'] = net.load['p_mw'] * 0.8
net.sgen['p_mw'] = net.sgen['p_mw'] * 0.8
net.gen['p_mw'] = net.gen['p_mw'] * 0.8
#scale down the load, generator and static generator since the sum of loads in the GBreducednetwork in 2013 is 56GW and the max peak demand in 2019 is 45GW based on National Grid

pp.runpp(net) #running the powerflow analysis with all lines intact
pp.to_excel(net,"GBreducednetworks.xlsx") #exporting the results into an excel document

results.at[:,"loading_percent"]=net.res_line["loading_percent"] #adding the loading percent for each line into the "results"
results.at[:,"initial_overload"]=net.res_line["loading_percent"] #adding the loading percent for each line into the "results"
results.loc[net.res_line.loading_percent <= 100, 'beforedrop_equal_or_greater_than_100'] = 0 
results.loc[net.res_line.loading_percent > 100, 'beforedrop_equal_or_greater_than_100'] = 1 
#creating a new column in the net.res_lines database for the number of overloaded lines
#setting the value equal to 0 if the line is not overloaded and 1 if it is

results.loc[net.res_line.loading_percent > 100, 'beforedrop_excess_over_100'] = (net.res_line.loading_percent - 100)
results.loc[net.res_line.loading_percent <= 100, 'beforedrop_excess_over_100'] = 0
#creating a new column in the net.res_lines database for the excess overload
#setting the value equal to the excess overload (if the line is overloaded) and 0 if it is not

x = range(0, len(net.res_line.index)) #setting a loop for dropping every line one at a time
for i in x:

    pp.drop_lines(net, [i]) #dropping a line one by one
    
    pp.runpp(net)  #running the powerflow analysis with the line dropped
    
    net.res_line.loc[net.res_line.loading_percent <= 100, 'afterdrop_equal_or_greater_than_100'] = 0 
    net.res_line.loc[net.res_line.loading_percent > 100, 'afterdrop_equal_or_greater_than_100'] = 1 
    #creating a new column in the net.res_lines database for the number of overloaded lines
    #setting the value equal to 0 if the line is not overloaded and 1 if it is
        
    net.res_line.loc[net.res_line.loading_percent > 100, 'excess_over_100'] = (net.res_line.loading_percent - 100)
    net.res_line.loc[net.res_line.loading_percent <= 100, 'excess_over_100'] = 0
    #creating a new column in the net.res_lines database for the excess overload
    #setting the value equal to the excess overload (if the line is overloaded) and 0 if it is not

    n_overload = np.sum(net.res_line.afterdrop_equal_or_greater_than_100) - np.sum(results.beforedrop_equal_or_greater_than_100)
    #creating a new variable to equal to the change in the number of overloaded lines as a result of dropping a given line
    
    excess_overload = np.sum(net.res_line.excess_over_100) - np.sum(results.beforedrop_excess_over_100)
    #creating a new variable to equal to the change in the excess overloads on the lines as a result of dropping a given line
    
    if results.loading_percent[i] >= 100 :
        n_overload = n_overload + 1
        excess_overload = excess_overload + (results.beforedrop_excess_over_100[i])
        #adding 1 if the dropped line was already overloaded, thus counting the number of overloaded line with the dropped line.
        
    results.at[i,"n_overload"]=n_overload #adding the number of overloaded lines to the "results"
    results.at[i,"excess_overload"]= excess_overload #adding the total excess overload on the lines to the "results"
    results.at[i,"overload_per_cable"]= excess_overload/n_overload
    
    net = pn.GBreducednetwork() #resetting the network (putting back the droppped line)
    net.load['p_mw'] = net.load['p_mw'] * 0.8
    net.sgen['p_mw'] = net.sgen['p_mw'] * 0.8
    net.gen['p_mw'] = net.gen['p_mw'] * 0.8
    #rescale down the load to 45 GW
    
print(results) #display the results Data Frame
print("Program finished") #signal the end of the program

The program started


C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarnin

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars


   n_overload excess_overload overload_per_cable  loading_percent  \
0           0      -0.0492794               -inf        29.525896   
1           0      -0.0492794               -inf        30.563889   
10          0      0.00534221                inf         4.050432   
11          0      0.00534221                inf         4.050432   
12          0       -0.450416               -inf        37.590981   
13          0       -0.450416               -inf        37.590981   
14          1         6.71343            6.71343        71.449823   
15          1         6.71343            6.71343        71.449823   
16          0      -0.0494063               -inf        16.189842   
17          0       -0.033063               -inf        13.039829   
18          0       -0.594373               -inf        29.456307   
19          0       -0.727608               -inf        46.106712   
2           0        -0.40579               -inf        61.019874   
20          0       0.0104095     